In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eedi-mining-misconceptions-in-mathematics/sample_submission.csv
/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv
/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv
/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv


In [2]:
# Import the required libraries
import pandas as pd  # For data manipulation and dataframe operations
import numpy as np   # For numerical operations
import os

In [3]:
# Set up Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = 'kevink963'
os.environ['KAGGLE_KEY'] = '78984c591112036c12a9075347e601b1'

In [4]:
# Reading datasets using pandas
train_df = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv')
test_df = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv')
misconception_df = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv')

In [5]:
# View the first 5 rows of the train dataset
print(train_df.head())

   QuestionId  ConstructId                                      ConstructName  \
0           0          856  Use the order of operations to carry out calcu...   
1           1         1612  Simplify an algebraic fraction by factorising ...   
2           2         2774            Calculate the range from a list of data   
3           3         2377  Recall and use the intersecting diagonals prop...   
4           4         3387  Substitute positive integer values into formul...   

   SubjectId                                        SubjectName CorrectAnswer  \
0         33                                             BIDMAS             A   
1       1077                    Simplifying Algebraic Fractions             D   
2        339  Range and Interquartile Range from a List of Data             B   
3         88                       Properties of Quadrilaterals             C   
4         67                          Substitution into Formula             A   

                          

In [6]:
# View the first 5 rows of the test dataset
print(test_df.head())

   QuestionId  ConstructId                                      ConstructName  \
0        1869          856  Use the order of operations to carry out calcu...   
1        1870         1612  Simplify an algebraic fraction by factorising ...   
2        1871         2774            Calculate the range from a list of data   

   SubjectId                                        SubjectName CorrectAnswer  \
0         33                                             BIDMAS             A   
1       1077                    Simplifying Algebraic Fractions             D   
2        339  Range and Interquartile Range from a List of Data             B   

                                        QuestionText            AnswerAText  \
0  \[\n3 \times 2+4-5\n\]\nWhere do the brackets ...  \( 3 \times(2+4)-5 \)   
1  Simplify the following, if possible: \( \frac{...              \( m+1 \)   
2  Tom and Katie are discussing the \( 5 \) plant...              Only\nTom   

              AnswerBText        

In [7]:
# View the first 5 rows of the misconception mapping dataset
print(misconception_df.head())

   MisconceptionId                                  MisconceptionName
0                0  Does not know that angles in a triangle sum to...
1                1  Uses dividing fractions method for multiplying...
2                2      Believes there are 100 degrees in a full turn
3                3  Thinks a quadratic without a non variable term...
4                4  Believes addition of terms and powers of terms...


In [8]:
# Checking for missing values in the train dataset
print(train_df.isnull().sum())

QuestionId            0
ConstructId           0
ConstructName         0
SubjectId             0
SubjectName           0
CorrectAnswer         0
QuestionText          0
AnswerAText           0
AnswerBText           0
AnswerCText           0
AnswerDText           0
MisconceptionAId    734
MisconceptionBId    751
MisconceptionCId    789
MisconceptionDId    832
dtype: int64


In [9]:
# Checking for missing values in the test dataset
print(test_df.isnull().sum())

QuestionId       0
ConstructId      0
ConstructName    0
SubjectId        0
SubjectName      0
CorrectAnswer    0
QuestionText     0
AnswerAText      0
AnswerBText      0
AnswerCText      0
AnswerDText      0
dtype: int64


In [10]:
# Checking for missing values in the misconception mapping dataset
print(misconception_df.isnull().sum())

MisconceptionId      0
MisconceptionName    0
dtype: int64


In [11]:
# Merging the train dataset with the misconception mapping based on MisconceptionId
train_merged = pd.merge(train_df, misconception_df, how='left', left_on='MisconceptionAId', right_on='MisconceptionId')

# Viewing the merged result
print(train_merged.head())

   QuestionId  ConstructId                                      ConstructName  \
0           0          856  Use the order of operations to carry out calcu...   
1           1         1612  Simplify an algebraic fraction by factorising ...   
2           2         2774            Calculate the range from a list of data   
3           3         2377  Recall and use the intersecting diagonals prop...   
4           4         3387  Substitute positive integer values into formul...   

   SubjectId                                        SubjectName CorrectAnswer  \
0         33                                             BIDMAS             A   
1       1077                    Simplifying Algebraic Fractions             D   
2        339  Range and Interquartile Range from a List of Data             B   
3         88                       Properties of Quadrilaterals             C   
4         67                          Substitution into Formula             A   

                          

In [12]:
# Distribution of correct answers in the train dataset
print(train_df['CorrectAnswer'].value_counts())

CorrectAnswer
C    488
A    482
B    461
D    438
Name: count, dtype: int64


In [13]:
# Distribution of misconceptions based on MisconceptionAId
print(train_df['MisconceptionAId'].value_counts())

MisconceptionAId
1214.0    18
1880.0    12
1379.0    11
1507.0    10
1990.0    10
          ..
1731.0     1
1788.0     1
1818.0     1
379.0      1
801.0      1
Name: count, Length: 740, dtype: int64


In [14]:
# Importing libraries for machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Separating features and target in the dataset
X = train_df[['QuestionId', 'ConstructId', 'SubjectId']]  # Features
y = train_df['MisconceptionAId']  # Target

In [15]:
# Removing rows with missing values in the target variable (y)
X = X[y.notna()]  # Select rows from X where y is not NaN
y = y[y.notna()]  # Select rows from y where y is not NaN

In [16]:
# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [17]:
# Making predictions on the validation set
y_pred = model.predict(X_val)

In [18]:
# Evaluating the model's accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f"Model accuracy: {accuracy}")

Model accuracy: 0.06167400881057269


In [19]:
# Making predictions on the test dataset
X_test = test_df[['QuestionId', 'ConstructId', 'SubjectId']]
test_predictions = model.predict(X_test)

In [20]:
# Creating a submission dataframe
submission_df = pd.DataFrame({
    'QuestionId_Answer': test_df['QuestionId'].astype(str) + '_A',
    'MisconceptionId': test_predictions
})

# Viewing the submission result
print(submission_df.head())

  QuestionId_Answer  MisconceptionId
0            1869_A            907.0
1            1870_A           2312.0
2            1871_A           2456.0


In [21]:
# Saving the submission to a CSV file
submission_df.to_csv('submission.csv', index=False)